In [ ]:
# Import standard libraries for data manipulation and also for Postgres 

import requests
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
import sqlalchemy 
import psycopg2

In [ ]:
github_api = "https://api.github.com"
gh_session = requests.Session()
gh_session.auth = ('username', 'API token')

In [ ]:
headers = {
             'Authorization': 'Token <API token>'
          }   

In [ ]:
def issues_of_repo_github(repo, owner, api):  #iterate through pages of respnse and append to issues
    issues = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/issues?page={}&per_page=100'.format(owner, repo, i)  #set page number
        issue_pg = gh_session.get(url = url,headers=headers,params=params)             # same as requests.get   
        issue_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in issue_pg.json()]   #simply unpacking json. json_normalize can be used here
        issue_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in issue_pg_list]        #
        issues = issues + issue_pg_list
        if 'Link' in issue_pg.headers: # if there exists another page then increment i or else dont
            if 'rel="next"' not in issue_pg.headers['Link']:
                next = False
        i = i + 1
    return issues

In [ ]:
def create_issues_df(repo, owner, api): #normalize json response to flat table 
    issues_list = issues_of_repo_github(repo, owner, api)
    return json_normalize(issues_list)

In [ ]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [ ]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [ ]:
%%time

repos=['reponame'] # mention repos you want to pull here
for r in repos:
    
    params = (
            ('type','issue'),
            ('state','all'),
            ('headers',headers)
        )
    issues = create_issues_df(r, 'company', github_api)

In [ ]:
issues['labels_name'] = issues['labels'].apply(lambda x: [y['name'] for y in x]) # extract lables from nested json 

In [ ]:
issues = issues.filter(['url','id', 'node_id', 'number', 'title', 'labels_name', 'state','created_at',
       'updated_at', 'closed_at', 'author_association','repo_name', 'owner', 'user.login',
       'user.id','user.type','assignee.login', 'assignee.id','assignee.type']) # Get desired columns only
issues

In [ ]:
unique_items = to_1D(issues["labels_name"]).value_counts() # Convert list to series

labels_bool = boolean_df(issues["labels_name"],unique_items.keys()) # Create a column for each label to that issue

issues = issues.join(labels_bool) # Join lables data to main
issues

In [ ]:
%%time

import sqlalchemy 
import psycopg2
engine = sqlalchemy.create_engine('postgres://username:password@ Server name:port/database name',
                                 use_batch_mode=True) 
con = engine.connect() #connect to pgsql db 
table_name='table' #set tablename 
issues.to_sql(table_name,con,if_exists='replace',index=False) #append data to table , replace earlier table with same name , if any
